# Knowledge Base를 활용한 검색

이 노트북에서는 Azure AI Search의 **Knowledge Base**를 사용하여 에이전틱 검색(Agentic Retrieval)을 수행합니다.

## 📋 목차
1. 환경 설정 및 라이브러리 설치
2. 환경 변수 설정
3. Knowledge Base 클라이언트 초기화
4. Knowledge Base 검색 수행
5. 검색 결과 상세 분석

## 사전 준비사항
- Azure AI Search 인덱스가 생성되어 있어야 합니다.
- Knowledge Source와 Knowledge Base가 Azure Portal에서 생성되어 있어야 합니다.
- `.env` 파일에 필요한 환경 변수가 설정되어 있어야 합니다.

## 1. 환경 설정 및 라이브러리 설치

Azure AI Search Knowledge Base SDK와 필요한 라이브러리를 설치합니다.

In [ ]:
# 필요한 패키지 설치
# azure-search-documents: Azure AI Search SDK (--pre 플래그로 최신 프리뷰 버전 설치)
# azure-identity: Azure 인증을 위한 SDK
# python-dotenv: 환경 변수 로드

%pip install --quiet --upgrade pip
%pip install --quiet python-dotenv azure-identity
%pip install --quiet azure-search-documents --pre --upgrade

## 2. 환경 변수 설정

`.env` 파일에서 Azure AI Search 및 Azure OpenAI 관련 환경 변수를 로드합니다.

### 인증 방식 선택
두 가지 인증 방식 중 선택할 수 있습니다:

| 방식 | 설명 | 환경 변수 |
|------|------|----------|
| **API 키** | 간단한 키 기반 인증 (권장: 로컬 개발) | `AZURE_SEARCH_ADMIN_KEY` |
| **DefaultAzureCredential** | Azure AD 기반 인증 (`az login` 필요) | - |

### 필요한 환경 변수 (`.env`)
```
# Azure AI Search
AZURE_SEARCH_SERVICE_ENDPOINT=https://your-search-service.search.windows.net
AZURE_SEARCH_ADMIN_KEY=your-search-admin-key-here
AZURE_KNOWLEDGE_BASE_NAME=kb-documents
AZURE_KNOWLEDGE_SOURCE_NAME=ks-documents
```

In [ ]:
import os
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv()

# Azure AI Search 엔드포인트
AZURE_SEARCH_SERVICE_ENDPOINT = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")

# Azure AI Search Admin 키 (API 키 인증 시 사용)
AZURE_SEARCH_ADMIN_KEY = os.getenv("AZURE_SEARCH_ADMIN_KEY")

# Knowledge Base 이름 (Azure Portal에서 생성한 Knowledge Base 이름)
AZURE_KNOWLEDGE_BASE_NAME = os.getenv("AZURE_KNOWLEDGE_BASE_NAME")

# Knowledge Source 이름 (Azure Portal에서 생성한 Knowledge Source 이름)
AZURE_KNOWLEDGE_SOURCE_NAME = os.getenv("AZURE_KNOWLEDGE_SOURCE_NAME")

# 환경 변수 확인
print("📍 환경 변수 설정 확인")
print("-" * 60)
print(f"Search Endpoint: {AZURE_SEARCH_SERVICE_ENDPOINT}")
print(f"Search Admin Key: {'설정됨' if AZURE_SEARCH_ADMIN_KEY else '설정 안됨'}")
print(f"Knowledge Base: {AZURE_KNOWLEDGE_BASE_NAME}")
print(f"Knowledge Source: {AZURE_KNOWLEDGE_SOURCE_NAME}")
print("-" * 60)

## 3. Knowledge Base 클라이언트 초기화

Azure 인증을 수행하고 Knowledge Base 클라이언트를 생성합니다.

두 가지 인증 방식 중 선택하여 사용하세요:
- **방법 1**: API 키 인증 (`az login` 불필요)
- **방법 2**: DefaultAzureCredential 인증 (`az login` 필요)

### 방법 1: API 키 인증 (az login 불필요)

API 키를 사용하여 인증합니다. `az login` 없이 바로 사용할 수 있습니다.

In [ ]:
# 방법 1: API 키 인증
# AzureKeyCredential을 사용하여 API 키로 인증
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.knowledgebases import KnowledgeBaseRetrievalClient
from azure.search.documents.knowledgebases.models import (
    KnowledgeBaseMessage, 
    KnowledgeBaseMessageTextContent, 
    KnowledgeBaseRetrievalRequest, 
    SearchIndexKnowledgeSourceParams,
)

# API 키가 설정되어 있는지 확인
if AZURE_SEARCH_ADMIN_KEY:
    # API 키로 클라이언트 생성
    credential = AzureKeyCredential(AZURE_SEARCH_ADMIN_KEY)
    kb_client = KnowledgeBaseRetrievalClient(
        endpoint=AZURE_SEARCH_SERVICE_ENDPOINT,
        knowledge_base_name=AZURE_KNOWLEDGE_BASE_NAME,
        credential=credential,
    )
    print("✅ API 키 인증으로 Knowledge Base 클라이언트 초기화 완료")
    print(f"📍 Endpoint: {AZURE_SEARCH_SERVICE_ENDPOINT}")
    print(f"📚 Knowledge Base: {AZURE_KNOWLEDGE_BASE_NAME}")
else:
    print("❌ API 키가 설정되지 않았습니다. .env 파일에 AZURE_SEARCH_ADMIN_KEY를 설정하세요.")

### 방법 2: DefaultAzureCredential 인증 (az login 필요)

Azure CLI로 로그인한 계정을 사용하여 인증합니다. 프로덕션 환경에서 권장됩니다.

In [ ]:
# 방법 2: DefaultAzureCredential 인증 (az login 필요)
# Azure CLI, 환경 변수, Managed Identity 등 다양한 인증 방식을 자동으로 시도
from azure.identity import DefaultAzureCredential
from azure.search.documents.knowledgebases import KnowledgeBaseRetrievalClient
from azure.search.documents.knowledgebases.models import (
    KnowledgeBaseMessage, 
    KnowledgeBaseMessageTextContent, 
    KnowledgeBaseRetrievalRequest, 
    SearchIndexKnowledgeSourceParams,
)

# DefaultAzureCredential 사용 시 아래 코드 실행
# 주의: 이 방법을 사용하려면 먼저 터미널에서 `az login` 실행 필요

# credential = DefaultAzureCredential()
# kb_client = KnowledgeBaseRetrievalClient(
#     endpoint=AZURE_SEARCH_SERVICE_ENDPOINT,
#     knowledge_base_name=AZURE_KNOWLEDGE_BASE_NAME,
#     credential=credential,
# )
# print("✅ DefaultAzureCredential로 Knowledge Base 클라이언트 초기화 완료")
# print(f"📍 Endpoint: {AZURE_SEARCH_SERVICE_ENDPOINT}")
# print(f"📚 Knowledge Base: {AZURE_KNOWLEDGE_BASE_NAME}")

print("ℹ️  DefaultAzureCredential 방식을 사용하려면 위 코드의 주석을 해제하세요.")
print("   사전에 터미널에서 `az login` 명령으로 Azure에 로그인해야 합니다.")

## 4. Knowledge Base 검색 수행

Knowledge Base에 질문을 전송하여 에이전틱 검색(Agentic Retrieval)을 수행합니다.

### 검색 요청 구성요소
- **messages**: 사용자 질문 메시지
- **knowledge_source_params**: Knowledge Source 설정
  - `include_references`: 참조 문서 포함 여부
  - `include_reference_source_data`: 참조 소스 데이터 포함 여부
  - `always_query_source`: 항상 소스 쿼리 수행 여부
- **include_activity**: 검색 활동 로그 포함 여부

In [ ]:
# 검색 쿼리 설정
search_query = "AI 지속 가능성에 대해서 알려줘"

# Knowledge Base 검색 요청 생성
request = KnowledgeBaseRetrievalRequest(
    # 사용자 메시지 설정
    messages=[
        KnowledgeBaseMessage(
            role="user",  # 사용자 역할
            content=[KnowledgeBaseMessageTextContent(text=search_query)]  # 질문 텍스트
        ),
    ],
    # Knowledge Source 파라미터 설정
    knowledge_source_params=[
        SearchIndexKnowledgeSourceParams(
            knowledge_source_name=AZURE_KNOWLEDGE_SOURCE_NAME,  # Knowledge Source 이름
            include_references=True,  # 참조 문서 포함
            include_reference_source_data=True,  # 참조 소스 데이터 포함
            always_query_source=True,  # 항상 소스 쿼리 수행
        )
    ],
    include_activity=True,  # 검색 활동 로그 포함
)

print(f"🔍 검색어: '{search_query}'")
print(f"\n{'='*60}")
print("[Knowledge Base 검색 수행 중...]")
print(f"{'='*60}\n")

# Knowledge Base 검색 실행
result = kb_client.retrieve(request)

print("✅ 검색 완료!")

In [ ]:
# 검색 결과 응답 출력
print(f"\n{'='*60}")
print("📝 [Knowledge Base 응답]")
print(f"{'='*60}\n")

# 응답 텍스트 출력 (AI 모델이 연결된 경우 생성된 답변)
if result.response:
    for response in result.response:
        if response.content:
            for content in response.content:
                print(content.text)
else:
    print("응답이 없습니다.")

## 5. 검색 결과 상세 분석

Knowledge Base 검색 결과에는 다음 정보가 포함됩니다:
- **response**: AI 모델이 생성한 답변
- **activity**: 검색 과정에서 수행된 활동 로그
- **references**: 답변 생성에 사용된 참조 문서 목록

In [ ]:
import json

# Activity 정보 출력 (검색 과정에서 수행된 활동 로그)
print(f"\n{'='*60}")
print("📊 [Activity - 검색 활동 로그]")
print(f"{'='*60}\n")

if result.activity:
    # Activity를 JSON 형식으로 변환하여 출력
    activity_content = json.dumps([a.as_dict() for a in result.activity], indent=2, ensure_ascii=False)
    print(activity_content)
else:
    print("활동 로그가 없습니다.")

In [ ]:
# References 정보 출력 (답변 생성에 사용된 참조 문서)
print(f"\n{'='*60}")
print("📚 [References - 참조 문서 목록]")
print(f"{'='*60}\n")

if result.references:
    # References를 JSON 형식으로 변환하여 출력
    references_content = json.dumps([r.as_dict() for r in result.references], indent=2, ensure_ascii=False)
    print(references_content)
else:
    print("참조 문서가 없습니다.")

In [ ]:
def print_references_summary(result):
    """
    참조 문서 목록을 보기 좋게 요약 출력합니다.
    
    Args:
        result: Knowledge Base 검색 결과
    """
    print(f"\n{'='*60}")
    print("📖 [참조 문서 요약]")
    print(f"{'='*60}\n")
    
    if not result.references:
        print("참조 문서가 없습니다.")
        return
    
    print(f"총 {len(result.references)}개의 참조 문서가 사용되었습니다.\n")
    
    for idx, ref in enumerate(result.references, 1):
        ref_dict = ref.as_dict()
        
        print(f"📄 참조 #{idx}")
        print(f"{'─'*60}")
        
        # 참조 ID
        if 'id' in ref_dict:
            print(f"   ID: {ref_dict['id']}")
        
        # 소스 데이터 (문서 내용)
        if 'source_data' in ref_dict and ref_dict['source_data']:
            source = ref_dict['source_data']
            
            # 제목
            if 'document_title' in source:
                print(f"   제목: {source['document_title']}")
            
            # 청크 내용 (일부만 출력)
            if 'content_text' in source:
                print(f"   내용: {source['content_text'][:30]}...")
        
        print()

# 참조 문서 요약 출력
print_references_summary(result)

## 6. 다른 질문으로 검색 테스트

다양한 질문으로 Knowledge Base 검색을 테스트해봅니다.

In [ ]:
def search_knowledge_base(query: str, include_details: bool = False):
    """
    Knowledge Base 검색을 수행하고 결과를 출력합니다.
    
    Args:
        query: 검색 질문
        include_details: 상세 정보(Activity, References) 출력 여부
    """
    # 검색 요청 생성
    request = KnowledgeBaseRetrievalRequest(
        messages=[
            KnowledgeBaseMessage(
                role="user",
                content=[KnowledgeBaseMessageTextContent(text=query)]
            ),
        ],
        knowledge_source_params=[
            SearchIndexKnowledgeSourceParams(
                knowledge_source_name=AZURE_KNOWLEDGE_SOURCE_NAME,
                include_references=True,
                include_reference_source_data=True,
                always_query_source=True,
            )
        ],
        include_activity=True,
    )
    
    print(f"🔍 검색어: '{query}'")
    print(f"{'='*60}\n")
    
    # 검색 실행
    result = kb_client.retrieve(request)
    
    # 응답 출력
    print("📝 [응답]")
    print(f"{'─'*60}")
    if result.response:
        for response in result.response:
            if response.content:
                for content in response.content:
                    print(content.text)
    else:
        print("응답이 없습니다.")
    
    # 상세 정보 출력 (선택적)
    if include_details:
        print_references_summary(result)
    
    print(f"\n{'='*60}\n")
    
    return result

In [ ]:
# 테스트 질문 1
result1 = search_knowledge_base("AI를 활용한 환경 보호 방안에 대해 알려줘", include_details=True)

In [ ]:
# 테스트 질문 2 (상세 정보 없이)
result2 = search_knowledge_base("Microsoft의 AI 서비스에 대해 설명해줘")

## 7. 정리

이 튜토리얼에서 배운 내용:

1. **Knowledge Base 클라이언트 설정**: `KnowledgeBaseRetrievalClient`를 사용한 클라이언트 초기화
2. **검색 요청 구성**: `KnowledgeBaseRetrievalRequest`로 검색 요청 생성
3. **에이전틱 검색 수행**: Knowledge Base를 통한 질의응답
4. **결과 분석**: 응답, Activity, References 정보 활용

### Knowledge Base의 장점
- 자연어 질문으로 문서 검색 가능
- AI 모델이 관련 문서를 기반으로 답변 생성
- 답변의 근거가 되는 참조 문서 제공
- 검색 과정의 투명성 (Activity 로그)